In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 64)
pd.set_option('display.max_rows', 100)
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.metrics import precision_score, recall_score
def calc_f2(label, predict):
    p = precision_score(label, predict)
    r = recall_score(label, predict)
    f2_score = 5*p*r / (4*p + r)
    return f2_score

In [3]:
homePath = 'game'
trainPath = os.path.join(homePath, 'train_fill.csv')
trainData = pd.read_csv(trainPath)
testPath = os.path.join(homePath, 'test_fill.csv')
testData = pd.read_csv(testPath)
submPath = os.path.join(homePath, 'submission.csv')
submData = pd.read_csv(submPath)

In [4]:
trainData['acc_now_delinq'] = list(map(int, trainData['acc_now_delinq']))

In [5]:
trainLabel = trainData['acc_now_delinq']
del trainData['acc_now_delinq']

In [6]:
trainCopy = trainData.copy()

In [7]:
trainCopy['annual_inc'].fillna(trainCopy['annual_inc'].mean(), inplace=True)
trainCopy['emp_title'].fillna(0, inplace=True)
# trainCopy['tot_coll_amt'].fillna(0, inplace=True)
testData['emp_title'].fillna(0, inplace=True)

In [8]:
testData.drop("acc_now_delinq", axis=1, inplace=True)

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# max_depth=13, min_samples_split=50, min_samples_leaf=20, max_features=9,
rnd_clf = RandomForestClassifier(n_estimators=110, min_samples_split=90, min_samples_leaf=15,max_depth=10,oob_score=True,max_features=10)

In [ ]:
rnd_clf.fit(trainCopy, trainLabel)

In [ ]:
del testData['acc_now_delinq']
rnd_clf_pred = rnd_clf.predict(testData)
rnd_clf_pred = pd.DataFrame(rnd_clf_pred)
testData['acc_now_delinq'] = rnd_clf_pred
testData['acc_now_delinq'].value_counts()

In [ ]:
rnd_pred = rnd_clf.predict(trainCopy)
rnd_f2 = calc_f2(trainLabel, rnd_pred)
rnd_f2

In [13]:
X, y = trainCopy[:100000], trainLabel[:100000]

In [ ]:
param_test1 =  {'max_features':range(8,14,2)}
rnd_1 = RandomForestClassifier(n_estimators=110,max_depth=10,min_samples_split=90, min_samples_leaf=15, oob_score=True,random_state=10)

In [ ]:
gsearch1 = GridSearchCV(estimator=rnd_1, param_grid=param_test1, scoring='roc_auc',cv=5)

In [ ]:
gsearch1.fit(X,y)

In [ ]:
print(gsearch1.best_params_)
print(gsearch1.best_score_)

# ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etr_clf = ExtraTreesClassifier(n_estimators=100, max_features='sqrt', n_jobs=2)

In [ ]:
etr_clf.fit(trainCopy[:100000], trainLabel[:100000])

In [ ]:
etr_clf_pred = etr_clf.predict(testData)
etr_clf_pred.sum()

In [ ]:
param_test2 = {'max_features':range(1,4,3)}
etr_1 = ExtraTreesClassifier(min_samples_split=102,max_depth=10, min_samples_leaf=11, n_estimators=61, n_jobs=-1)

In [ ]:
gsearch2 = GridSearchCV(estimator=etr_1, param_grid=param_test2, scoring='roc_auc',cv=5)

In [ ]:
gsearch2.fit(X,y)

In [ ]:
print(gsearch2.best_params_)
print(gsearch2.best_score_)

# GradientBoostingClassifier

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation, metrics
gbdt = GradientBoostingClassifier(n_estimators= 40, learning_rate=0.1, min_samples_split=320, min_samples_leaf=7, max_depth=13, 
                                 max_features='sqrt', subsample=0.8)

In [36]:
gbdt.fit(trainCopy[:100000], trainLabel[:100000])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=13,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=7, min_samples_split=320,
              min_weight_fraction_leaf=0.0, n_estimators=40,
              presort='auto', random_state=None, subsample=0.8, verbose=0,
              warm_start=False)

In [37]:
gbdt_pred = gbdt.predict(testData)

In [38]:
gbdt_pred.sum()

189

In [10]:
param_test3 = {'max_depth':range(3,14,2), 'min_samples_split':range(100,120,2)}
gbdt_1 = GradientBoostingClassifier(n_estimators= 40, learning_rate=0.1,min_samples_leaf=7,
                                 max_features='sqrt', subsample=0.7, random_state=10)

In [14]:
gsearch3 = GridSearchCV(estimator=gbdt_1,param_grid=param_test3, scoring='roc_auc',iid=False,cv=5)
gsearch3.fit(X,y)

KeyboardInterrupt: 

In [ ]:
print(gsearch3.best_params_)
print(gsearch3.best_score_)

# 投票模型

In [ ]:
from sklearn.ensemble import VotingClassifier
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(learning_rate =0.05,
                     n_estimators=1000,
                     max_depth=3,
                     min_child_weight=1,
                     gamma=0.1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'binary:logistic',
                     nthread=4,
                     reg_alpha=0.001,
                     reg_lambda=0.001,
                     scale_pos_weight=1)

import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(learning_rate=0.1,
                            boosting_type='gbdt',
                            objective='binary',
                            n_estimators=1000,
                            metric='auc',
                            max_depth=3,
                            num_leaves=5,
                            subsample=0.7,
                            colsample_bytree=0.7,
                            min_data_in_leaf=450,
                            feature_fraction=0.7,
                            bagging_fraction=0.7,
                            bagging_freq=6,
                            lambda_l1=1,
                            lambda_l2=0.001,
                            min_gain_to_split=0.265,
                            verbose=5,
                            is_unbalance=True)

from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier(learning_rate=0.05, min_samples_split=320, min_samples_leaf=7, max_depth=7, 
                                 max_features='sqrt', subsample=0.7, random_state=10)

vot = VotingClassifier(estimators=[('xgb', xgb), ('lgb', lgb_clf)], voting='soft')

In [ ]:
vot.fit(trainCopy[:100000], trainLabel[:100000])

In [ ]:
del testData['acc_now_delinq']
vot_pred = vot.predict(testData)
vot_pred = pd.DataFrame(vot_pred)
testData['acc_now_delinq'] = vot_pred
testData['acc_now_delinq'].value_counts()

In [ ]:
submData['acc_now_delinq'] = vot_pred

In [ ]:
submData.to_csv(os.path.join(homePath, 'subm_fill.csv'), index=False)

In [ ]:
vot_s_pred = vot.predict(trainCopy)
vot_s_f2 = calc_f2(trainLabel, vot_s_pred)
vot_s_f2